# Обучение FastText

В бейзлайне предполагается использовать векторы из библиотеки FastText, которые в данном ноутбуке обучатся на домен чеков

In [1]:
from gensim.models.fasttext import FastText
import pandas as pd
import numpy as np

Выгрузим все чеки из размеченного датасета для обучения, а также из тестового датасета

In [2]:
train_df = pd.read_csv("data/train_supervised_dataset.csv").fillna("")
uns_train_df = pd.read_csv("data/train_unsupervised_dataset.csv").fillna("")
test_df = pd.read_csv("data/test_dataset.csv")
names = pd.concat((train_df[["name"]], test_df, uns_train_df[["name"]])).reset_index(drop=True)
names

,name,id
0,Petmax Бантик леопард с красн розой 2шт,NaN
1,87191 Бусы для елки шарики_87191,NaN
2,Футболка Piazza Italia WR011446881,NaN
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN
4,Одежда (вес) 1500,NaN
...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN


In [3]:
brands = train_df[['brand']].values

In [4]:
np.unique(brands)

array(['', '1toy', '21 век', ..., 'ячменное поле', 'яшкино', '№1 school'],
      dtype=object)

In [5]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [6]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

names_extractor = NamesExtractor(morph_vocab)

In [7]:
def lemmatize(words):
    text = ' '.join(words)
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    
    words = []
    for token in doc.tokens:
        words.append(token.lemma)
    return words

In [8]:
import re

from cyrtranslit import to_cyrillic

def preprocess_text(text):
    
    text = re.sub('\d+', '<NUM>', text)  # replace numbers to 0
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = re.sub(r'[^\w\s]', ' ', text)  # remove punctuation
#     text = text.replace('0', '')
    
    words = []
    for w in text.lower().split():
        if bool(re.search('[a-z]', w)) and bool(re.search('[а-я]', w)):
            words.append(to_cyrillic(w))
        else:
            words.append(w)
#     words = lemmatize(words)
    return words

In [9]:
text = names['name'].sample().values[0]

In [10]:
%%time
names["tokens"] = names["name"].apply(preprocess_text)
names

CPU times: user 8 µs, sys: 20 µs, total: 28 µs
Wall time: 56 µs


,name,id,tokens
0,Petmax Бантик леопард с красн розой 2шт,NaN,"[petmax, бантик, леопард, с, красн, розой, num..."
1,87191 Бусы для елки шарики_87191,NaN,"[num, бусы, для, елки, шарики_, num]"
2,Футболка Piazza Italia WR011446881,NaN,"[футболка, piazza, italia, wr, num]"
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN,"[num, yi, num, num, x, one, заколка, для, воло..."
4,Одежда (вес) 1500,NaN,"[одежда, вес, num]"
...,...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN,"[f, num, спонж, д, макияжа, фигурный, шт]"
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN,"[num, num, num, num, дифф, arom, num]"
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN,"[матрас, надувной, num, num, см, tropical, bir..."
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN,"[пододеяльник, стм, страйп, num, сп, размер, n..."


In [11]:
# names.to_csv('data/natasha_preprocc_names.csv', index=False)

In [11]:
print(names['name'].values)

['Petmax Бантик леопард с красн розой 2шт'
 '87191 Бусы для елки шарики_87191' 'Футболка Piazza Italia WR011446881'
 ... 'Матрас надувной 540*74см Tropical Bird запл.д/ремонта 5 дизайна'
 'пододеяльник СТМ Страйп 3-сп, размер: 796х185см, страйп-сатин, 736% хлопок, 158гр/м1, сиреневый'
 'Пакет ламинированный вертикальный ?Чудесных мгновений!?, 52 ? 44']


In [12]:
names

,name,id,tokens
0,Petmax Бантик леопард с красн розой 2шт,NaN,"[petmax, бантик, леопард, с, красн, розой, num..."
1,87191 Бусы для елки шарики_87191,NaN,"[num, бусы, для, елки, шарики_, num]"
2,Футболка Piazza Italia WR011446881,NaN,"[футболка, piazza, italia, wr, num]"
3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,NaN,"[num, yi, num, num, x, one, заколка, для, воло..."
4,Одежда (вес) 1500,NaN,"[одежда, вес, num]"
...,...,...,...
1029995,F-2296 Спонж д/макияжа фигурный (шт),NaN,"[f, num, спонж, д, макияжа, фигурный, шт]"
1029996,4 5702737510597 69.88 Дифф/Arom/1601,NaN,"[num, num, num, num, дифф, arom, num]"
1029997,Матрас надувной 540*74см Tropical Bird запл.д/...,NaN,"[матрас, надувной, num, num, см, tropical, bir..."
1029998,"пододеяльник СТМ Страйп 3-сп, размер: 796х185с...",NaN,"[пододеяльник, стм, страйп, num, сп, размер, n..."


In [13]:
class SentenceGenerator(object):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __iter__(self):
        for index, row in self.dataframe.iterrows():
            yield row['tokens']

sentences = SentenceGenerator(names)

Обученные векторы положим в файл `fasttext.model` для использования в моделях

In [15]:
%%time
FastText(names["tokens"], vector_size=128, window=5, min_count=5).save("fasttext_models/fasttext_128.model")

CPU times: user 3min 56s, sys: 26.7 s, total: 4min 22s
Wall time: 1min 50s
